In [1]:
# have to tell it where Gurobi and the license file are on my machine.

ENV["GUROBI_HOME"] = "/Library/gurobi1000/macos_universal2"
ENV["GRB_LICENSE_FILE"] = "/Library/gurobi1000/gurobi.lic"
import Pkg
Pkg.add("Gurobi")
Pkg.build("Gurobi")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
    Building Gurobi → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/82a44a86f4dc4fa4510c9d49b0a74d3d73914d5c/build.log`


## Example: ClothCo
ClothCo is capable of manufacturing three types of clothing: shirts, shorts, and pants. Each type of clothing requires that ClothCo have the appropriate type of machine available. The machines can be rented at a fixed weekly cost. The manufacture of each type of clothing also requires some amount of cloth and labor, and returns some profit, indicated below. Each week, 150 hours of labor and 160 sq yd of cloth are available. How should ClothCo tune its production to maximize profits? Note: If we don't produce a particular item, we can save on the rental cost!

| Clothing item | Labor per item | Cloth per item | Profit per item | Machine rental |
|:------:|:-------:|:-:|:---:|:--------:|
| Shirt  | 3 hours | 4 | \$6 | \$200/wk |
| Shorts | 2 hours | 3 | \$4 | \$150/wk |
| Pants  | 6 hours | 4 | \$7 | \$100/wk |

In [2]:
using JuMP, Gurobi

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, x[1:3] >= 0)
@variable(m, z[1:3], Bin)

@constraint(m, 3x[1] + 2x[2] + 6x[3] <= 150)  # labor budget
@constraint(m, 4x[1] + 3x[2] + 4x[3] <= 160)  # cloth budget
@constraint(m, x .<= 54*z)                   # if x>0 then z=1

@objective(m, Max, 6x[1] + 4x[2] + 7x[3] - 200z[1] - 150z[2] - 100z[3])
#@objective(m, Max, 6x[1] + 4x[2] + 7x[3])

optimize!(m)

xopt = value.(x)
println(xopt[1], " shirts")
println(xopt[2], " shorts")
println(xopt[3], " pants" )
println()
println("\$", objective_value(m), " of net profit")

[ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]


Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-15
0.0 shirts
0.0 shorts
25.0 pants

$75.0 of net profit


In [3]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, x[1:3] >= 0)
@variable(m, z[1:3], Bin)

@constraint(m, 3x[1] + 2x[2] + 6x[3] <= 150)  # labor budget
@constraint(m, 4x[1] + 3x[2] + 4x[3] <= 160)  # cloth budget
@constraint(m, x .<= 54*z)                   # if x>0 then z=1

@objective(m, Max, 6x[1] + 4x[2] + 7x[3] - 200z[1] - 150z[2] - 100z[3])
#@objective(m, Max, 6x[1] + 4x[2] + 7x[3])

optimize!(m)

xopt = value.(x)
println(xopt[1], " shirts")
println(xopt[2], " shorts")
println(xopt[3], " pants" )
println()
println("\$", objective_value(m), " of net profit")

0.0 shirts
0.0 shorts
25.0 pants

$75.0 of net profit
